# New Section

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import col, sum, avg

In [ ]:
spark = SparkSession.builder \
    .appName("CustomerTransactions") \
    .getOrCreate()

In [ ]:
data = [
    (1, "2024-01-01", 100.0),
    (1, "2024-01-03", 50.0),
    (1, "2024-01-07", 200.0),
    (2, "2024-01-01", 150.0),
    (2, "2024-01-04", 300.0),
]
columns = ["customer_id", "transaction_date", "amount"]
transactions = spark.createDataFrame(data, schema=columns)

In [ ]:
from pyspark.sql.functions import to_date
transactions = transactions.withColumn("transaction_date", to_date("transaction_date"))

In [ ]:
cumulative_window = Window.partitionBy("customer_id").orderBy("transaction_date") \
                          .rowsBetween(Window.unboundedPreceding, Window.currentRow)

In [ ]:
rolling_window = Window.partitionBy("customer_id").orderBy("transaction_date") \
                       .rangeBetween(-6, 0)

In [ ]:
transactions = transactions.withColumn(
    "cumulative_amount",
    sum("amount").over(cumulative_window)
)

In [ ]:
rolling_window = Window.partitionBy("customer_id").orderBy("transaction_date").rowsBetween(-6, 0)
transactions = transactions.withColumn(
    "rolling_avg_amount",
    avg("amount").over(rolling_window)
)

In [ ]:
result_df = transactions.select(
    "customer_id",
    "transaction_date",
    "amount",
    "cumulative_amount",
    "rolling_avg_amount"
)

In [ ]:
result_df.show()

+-----------+----------------+------+-----------------+------------------+
|customer_id|transaction_date|amount|cumulative_amount|rolling_avg_amount|
+-----------+----------------+------+-----------------+------------------+
|          1|      2024-01-01| 100.0|            100.0|             100.0|
|          1|      2024-01-03|  50.0|            150.0|              75.0|
|          1|      2024-01-07| 200.0|            350.0|116.66666666666667|
|          2|      2024-01-01| 150.0|            150.0|             150.0|
|          2|      2024-01-04| 300.0|            450.0|             225.0|
+-----------+----------------+------+-----------------+------------------+

